In [1]:
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import ArxivQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
#pip install arxiv

  Using cached sgmllib3k-1.0.0-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [arxiv]
Note: you may need to restart the kernel to use updated packages.


In [4]:


# -------------------------------------------------------
# LLM
# -------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# -------------------------------------------------------
# TOOLS
# -------------------------------------------------------

@tool
def decompose_topic(topic: str) -> str:
    """Decompose a research topic into subtopics."""
    return llm.invoke(
        f"Break the topic '{topic}' into 4–6 research subtopics."
    ).content


@tool
def summarize(text: str) -> str:
    """Summarize research content into key insights."""
    return llm.invoke(
        f"Summarize the following content into key insights:\n{text}"
    ).content


# Built-in research tools
arxiv_tool = ArxivQueryRun()
tavily_tool = TavilySearchResults(max_results=3)

# -------------------------------------------------------
# AGENT (Zero-shot ReAct)
# -------------------------------------------------------
agent = create_react_agent(
    model=llm,
    tools=[decompose_topic, arxiv_tool, tavily_tool, summarize]
)

# -------------------------------------------------------
# ONE-STEP INVOCATION
# -------------------------------------------------------
research_topic = "Agentic AI in Healthcare"

result = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": f"""
Research the following topic and produce a concise research brief.

Steps you should decide internally:
- Decompose the topic
- Search Arxiv and the web
- Summarize findings
- Present a structured report

Topic: {research_topic}
"""
        }
    ]
})

print(result)


/var/folders/m8/lr5c6v793qbcszm54dsf6y440000gn/T/ipykernel_3879/2956840812.py:28: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=3)
/var/folders/m8/lr5c6v793qbcszm54dsf6y440000gn/T/ipykernel_3879/2956840812.py:33: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


{'messages': [HumanMessage(content='\nResearch the following topic and produce a concise research brief.\n\nSteps you should decide internally:\n- Decompose the topic\n- Search Arxiv and the web\n- Summarize findings\n- Present a structured report\n\nTopic: Agentic AI in Healthcare\n', additional_kwargs={}, response_metadata={}, id='a3783648-369e-4a65-bcca-f9cee8534905'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 250, 'total_tokens': 269, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-CmX238DHxw12wPsbOaUsmvKRdf3SH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--b174ea0f-3bb4

In [5]:
def extract_final_research_text(agent_output: dict) -> str:
    """
    Extract the final research brief text from a LangGraph agent output.
    """

    messages = agent_output.get("messages", [])

    # Traverse messages in reverse to find the last AIMessage with content
    for msg in reversed(messages):
        if msg.__class__.__name__ == "AIMessage":
            content = getattr(msg, "content", "")
            if content and content.strip():
                return content.strip()

    return "No final research content found."
final_text = extract_final_research_text(result)
print(final_text)


### Research Brief: Agentic AI in Healthcare

#### 1. Introduction
Agentic AI refers to autonomous artificial intelligence systems capable of performing complex medical tasks, making clinical decisions, and interacting within healthcare environments without constant human intervention. This research brief explores the multifaceted role of agentic AI in healthcare, focusing on its applications, benefits, challenges, and ethical considerations.

#### 2. Decomposed Subtopics
- **Clinical Decision Support Systems**
- **Patient Engagement and Personalized Care**
- **Ethical and Regulatory Considerations**
- **AI in Medical Imaging and Diagnostics**
- **AI-Driven Predictive Analytics for Population Health**
- **Interdisciplinary Collaboration and Implementation Challenges**

#### 3. Key Findings

**3.1 Clinical Decision Support Systems**
- **Promise of Machine Learning**: Machine learning has significant potential to improve healthcare outcomes, particularly through clinical decision support